In [1]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/catanaranjo/fraudDetection/master/dev.csv") 
dfo = pd.read_csv("https://raw.githubusercontent.com/catanaranjo/fraudDetection/master/oot0.csv")

In [2]:
df['dataset'] = 'train'
dfo['dataset'] = 'test'
dfo['ob_target'] = 0
trainTestDF= df.append(dfo)

In [3]:
numericList = []
for i in range(65,82):
    numericList.append('if_var_' + str(i))

In [4]:
import numpy as np
import warnings; warnings.simplefilter('ignore')
def change_outliers(dataframe):
    list = []
    list = numericList
    for column in list:
        q1 = dataframe[column].quantile(0.25)
        q3 = dataframe[column].quantile(0.75)
        iqr = q3-q1 #Interquartile range
        highest = q3 + 1.5*iqr
        lowest = q1 - 1.5*iqr
        median = dataframe.loc[dataframe[column]<q3 + 1.5*iqr, column].median()
        outliersUp = (dataframe[column]).abs() > round(q3 + 1.5*iqr)
        outliersDown = (dataframe[column]).abs() < round(q1 - 1.5*iqr)
        dataframe[column][outliersUp] = np.nan
        dataframe[column].fillna(highest, inplace=True)
        dataframe[column][outliersDown] = np.nan
        dataframe[column].fillna(lowest, inplace=True)

numericDF = trainTestDF[numericList]
change_outliers(numericDF)

In [5]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8,random_state=42)
transformed = pca.fit_transform(numericDF)

pcaDF = pd.DataFrame(data = transformed
             , columns = ['PCA1', 'PCA2', 'PCA3','PCA4','PCA5','PCA6','PCA7','PCA8'])
pcaDF.reset_index(inplace=True)
trainTestDF = trainTestDF.join(pcaDF)

In [6]:
categorical_transformed_trainDF = trainTestDF[trainTestDF['dataset']=='train']
categorical_transformed_testDF = trainTestDF[trainTestDF['dataset']=='test']

# y_train = trainTestDF['ob_target']
y_train = categorical_transformed_trainDF['ob_target']

drops = ['ob_target','dataset']
categorical_ohe_train = categorical_transformed_trainDF.loc[:,~categorical_transformed_trainDF.columns.isin(drops)]
categorical_ohe_test = categorical_transformed_testDF.loc[:,~categorical_transformed_testDF.columns.isin(drops)]

X_train = pd.get_dummies(categorical_ohe_train) 
X_test = pd.get_dummies(categorical_ohe_test)
X_train.head()

,id,ib_var_1,ib_var_2,ib_var_3,ib_var_4,ib_var_5,ib_var_6,ib_var_7,ib_var_8,ib_var_9,...,if_var_81,index,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8
0,1,0,1,0,0,1,1,0,0,0,...,1,0,760.620588,1560.193129,2501.618834,-8.365096,2.155543,-4.398573,0.126588,-1.297023
1,2,0,1,0,0,0,1,0,0,0,...,4,1,-4243.644181,-644.567356,-487.223580,-5.249111,-0.195600,-3.760637,1.292445,-0.423682
2,3,0,0,0,0,1,1,0,0,0,...,2,2,373.306564,1027.206123,-1035.542924,11.381250,2.184368,-1.336323,1.177665,-2.237031
3,4,0,1,0,1,1,1,0,0,0,...,3,3,-3072.434904,-1877.732155,2916.527233,-8.005407,0.779459,-4.956250,-4.217975,0.226892
4,5,0,0,0,0,0,1,0,0,0,...,3,4,-5148.297300,2163.786424,-602.197494,0.259998,-3.179233,-2.533937,-7.518656,-2.485807


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

baseline_rf = RandomForestClassifier(n_jobs=-1,random_state=42,n_estimators=4000,criterion='entropy',max_features='log2',
                                    bootstrap=True, min_samples_split=3,min_samples_leaf=1,oob_score=True)

baseline_rf.fit(X_train,y_train)
y_pred_rf = baseline_rf.predict_proba(X_train)[:,1]

 
gini_score = 2*roc_auc_score(y_train, y_pred_rf)-1
print ("GINI DEVELOPMENT=", gini_score)

GINI DEVELOPMENT= 1.0
